# Lab: Building a Cognitive Language Agent with Memory, Tools, and the Decision Cycle

Welcome! In this lab, you'll build a simple agent that can remember, use tools, and act autonomously using AWS Bedrock's Claude Sonnet model.

## Learning Objectives

- Set up AWS Bedrock and Claude Sonnet for agent reasoning
- Implement agent memory using a vector store (FAISS)
- Build and register tools for agent use
- Implement the agent decision cycle (Observe → Plan → Act)
- Experiment with agentic vs. single-shot LLM workflows

## 1. Environment Setup

This notebook works on both [MyBinder](https://mybinder.org/) and AWS SageMaker.

**AWS Credentials:**
- **MyBinder:** Set your credentials as environment variables in a cell (do NOT share credentials).
- **SageMaker:** Credentials are pre-configured.

**Dependencies:**  
- `boto3` (AWS SDK)
- `faiss-cpu` (vector store)
- `sentence-transformers` (embeddings)
- `matplotlib`, `pandas` (optional, for visualization)

In [3]:
# If running locally or on MyBinder, uncomment the next line:
# !pip install boto3 faiss-cpu sentence-transformers matplotlib pandas
import os
import boto3
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import pandas as pd
import matplotlib.pyplot as plt

/Users/arjunasoknair/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# MyBinder users: set your credentials here (do NOT share real keys)
# os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_ACCESS_KEY'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_SECRET_KEY'
# os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'  # or your region

region = os.environ.get('AWS_DEFAULT_REGION', 'us-west-2')
bedrock = boto3.client('bedrock-runtime', region_name=region)

In [5]:
# Set the Claude model ID (update as needed for your account)
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'  # or 'anthropic.claude-3-5-sonnet-20241022-v2:0'

def call_bedrock_claude(prompt, system_prompt=None, max_tokens=512):
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.2,
        "top_k": 1
    }
    if system_prompt:
        request_body["system"] = system_prompt

    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body),
        contentType='application/json'
    )
    result = json.loads(response['body'].read())
    return result['content'][0]['text'].strip()

## 2. Implementing Agent Memory with a Vector Store

We'll use FAISS for fast similarity search and `sentence-transformers` for embeddings.
- **Working memory:** Current context
- **Episodic memory:** Vector store of past user/agent turns